In [4]:
import numpy as np
import torch 
import torch.nn as nn 
from torch.distributions import Categorical
import matplotlib.pyplot as plt 
from matplotlib import rcParams
rcParams['font.size'] = 24
rcParams['figure.figsize'] = (16, 8)
from tqdm import tqdm 

import importlib 
import ipywidgets
from ipywidgets import interact
from IPython.display import Image
import IPython

from rllib.dataset.datatypes import Observation
from rllib.util.utilities import get_entropy_and_log_p

from rllib.util.training.agent_training import train_agent
from rllib.environment import GymEnvironment
from rllib.environment.mdps import EasyGridWorld
from rllib.policy import TabularPolicy
from rllib.value_function import TabularQFunction, TabularValueFunction
from rllib.util.parameter_decay import ExponentialDecay

import warnings
warnings.filterwarnings('ignore')

In [5]:
def extract_policy(q_function):
    """Extract a policy from the q_function."""
    policy = TabularPolicy(num_states=q_function.num_states,
                           num_actions=q_function.num_actions)
    for state in range(policy.num_states):
        q_val = q_function(torch.tensor(state).long())
        action = torch.argmax(q_val)

        policy.set_value(state, action)

    return policy

def integrate_q(q_function, policy):
    value_function = TabularValueFunction(num_states=q_function.num_states)
    for state in range(policy.num_states):
        state = torch.tensor(state).long()
        pi = Categorical(logits=policy(state))
        value = 0
        for action in range(policy.num_actions):
            value += pi.probs[action] * \
                q_function(state, torch.tensor(action).long())

        value_function.set_value(state, value)

    return value_function


environment = EasyGridWorld()
Image("images/grid_world.png")

# Plotters
def plot_value_function(value_function, ax):
    ax.imshow(value_function)
    rows, cols = value_function.shape
    for i in range(rows):
        for j in range(cols):
            ax.text(j, i, f"{value_function[i, j]:.1f}",
                    ha="center", va="center", color="w")

def policy2str(policy):
    left = u'\u2190'
    right = u'\u2192'
    up = u'\u2191'
    down = u'\u2193'
    policy_str = ""
    if 0 == policy:
        policy_str += down 
    if 1 == policy:
        policy_str += up 
    if 2 == policy:
        policy_str += right
    if 3 == policy:
        policy_str += left
    return policy_str

def plot_value_function(value_function, ax, title=None):
    if title is not None:
        ax.set_title(title)
    ax.imshow(value_function)
    rows, cols = value_function.shape
    for row in range(rows):
        for col in range(cols):
            ax.text(row, col, f"{value_function[col, row]:.1f}", ha="center", va="center", color="w", fontsize=24)

def plot_policy(policy, ax, title):
    if title is not None:
        ax.set_title(title)
    rows, cols = policy.shape
    ax.imshow(np.zeros((rows, cols)))
    for row in range(environment.height):
        for col in range(environment.width):
            ax.text(col, row, policy2str(policy[row, col]), ha="center", va="center", color="r", fontsize=24)


def plot_value_and_policy(value_function, policy, title_value=None, title_policy=None):
    fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(20, 8))

    plot_value_function(value_function, axes[0], title=title_value)
    plot_policy(policy, axes[1], title=title_policy)
    

def plot_value_advantage_and_policy(value_function, advantage, policy, title_value=None, title_advantage=None, title_policy=None):
    fig, axes = plt.subplots(ncols=3, nrows=1, figsize=(20, 8))

    plot_value_function(value_function, axes[0], title=title_value)
    plot_value_function(advantage, axes[1], title=title_advantage)
    plot_policy(policy, axes[2], title=title_policy)
    
    
def init_value_function(num_states, terminal_states=None):
    """Initialize value function."""
    value_function = TabularValueFunction(num_states=num_states)
    terminal_states = [] if terminal_states is None else terminal_states
    for terminal_state in terminal_states:
        value_function.set_value(terminal_state, 0)

    return value_function


def build_mrp_matrices(environment, policy):
    mrp_kernel = np.zeros((environment.num_states, 1, environment.num_states))
    mrp_reward = np.zeros((environment.num_states, 1))

    for state in range(environment.num_states):
        state = torch.tensor(state).long()
        policy_ = Categorical(logits=policy(state))

        for a, p_action in enumerate(policy_.probs):
            for transition in environment.transitions[(state.item(), a)]:
                with torch.no_grad():
                    p_ns = transition["probability"]
                    mrp_reward[state, 0] += p_action * p_ns * transition["reward"]
                    mrp_kernel[state, 0, transition["next_state"]
                               ] += p_action * p_ns

    return mrp_kernel, mrp_reward


def kernelreward2transitions(kernel, reward):
    """Transform a kernel and reward matrix into a transition dicitionary."""
    transitions = defaultdict(list)

    num_states, num_actions = reward.shape

    for state in range(num_states):
        for action in range(num_actions):
            for next_state in np.where(kernel[state, action])[0]:
                transitions[(state, action)].append(
                    {
                        "next_state": next_state,
                        "probability": kernel[state, action, next_state],
                        "reward": reward[state, action],
                    }
                )

    return transitions

def linear_system_policy_evaluation(environment, policy, gamma, value_function=None):
    """Evaluate a policy in an MDP solving the system bellman of equations.

    V = r + gamma * P * V
    V = (I - gamma * P)^-1 r
    """

    if value_function is None:
        value_function = init_value_function(environment.num_states)

    kernel, reward = build_mrp_matrices(environment=environment, policy=policy)

    A = torch.eye(environment.num_states) - gamma * kernel[:, 0, :]
    # torch.testing.assert_allclose(A.inverse() @ A, torch.eye(model.num_states))
    vals = A.inverse() @ reward[:, 0]
    for state in range(environment.num_states):
        value_function.set_value(state, vals[state].item())

    return value_function

def policy_iteration_step(environment, policy, value_function, gamma):
    value_function = linear_system_policy_evaluation(environment, policy, gamma)
    policy_stable = True
    for state in range(environment.num_states):

        value_ = torch.zeros(environment.num_actions)
        for action in range(environment.num_actions):
            value_estimate = 0
            for transition in environment.transitions[(state, action)]:
                next_state = torch.tensor(transition["next_state"]).long()
                reward = torch.tensor(transition["reward"]).double()
                value_estimate += transition["probability"] * (
                    reward + gamma * value_function(next_state).item()
                )

            value_[action] = value_estimate

        state = torch.tensor(state).long()
        old_policy = policy(state)
        old_action = torch.argmax(old_policy)

        action = torch.argmax(value_)
        policy.set_value(state, action)

        policy_stable &= (action == old_action).all().item()
    
    return value_function, policy_stable 


def policy_iteration(environment, gamma, max_iter=10, value_function=None, policy=None):
    """Implement Policy Iteration algorithm.

    Parameters
    ----------
    gamma: float.
        discount factor.

    References
    ----------
    Sutton, R. S., & Barto, A. G. (2018). Reinforcement learning: An introduction.
    MIT press.
    Chapter 4.3

    """
    if value_function is None:
        value_function = init_value_function(environment.num_states)
    if policy is None:
        policy = TabularPolicy(num_states=environment.num_states, num_actions=environment.num_actions)

    for num_iter in range(max_iter):
        # Evaluate the policy. 
        value_function, policy_stable = policy_iteration_step(environment, policy, value_function, gamma)
        
        if policy_stable:
            break
    return value_function, policy, num_iter

# REINFORCE 

In [7]:
def reinforce(gamma, alpha, baseline):
    output = ipywidgets.Output()
    global policy 
    policy = TabularPolicy(num_states=environment.num_states,
                           num_actions=environment.num_actions)
    nn.init.ones_(policy.nn.head.weight)

    episode_horizon = 20

    def step(num_episodes):
        global policy 
        for i_episode in range(num_episodes):
            # The initial distribution plays a big role.
            state = environment.reset()
            trajectory = []
            for i in range(episode_horizon):
                pi = Categorical(logits=policy(torch.tensor(state).long()))
                action = pi.sample().item()

                next_state, reward, done, info = environment.step(action)
                trajectory.append([state, action, reward, next_state])

                state = next_state

            # Calculate sum of discounted returns
            cum_returns, rewards = 0, []
            returns, grad_log_prob_action = [], []
            for state, action, reward, next_state in trajectory[::-1]:
                reward = reward[0]  # get the reward number.
                cum_returns = reward + gamma * cum_returns  # why?
                returns.insert(0, cum_returns)
                grad_log_prob_action.insert(0, 1)  # In this case the gradient is very easy. 

            # Normalize returns for variance reduction.
            returns = torch.tensor(returns)
            if baseline:
                returns = (returns - returns.mean()) / (returns.std() + 1e-12)
            policy_grad = []
            for grad_log_prob, ret in zip(grad_log_prob_action, returns):
                policy_grad.append(-grad_log_prob * ret)

            for (state, action, reward, next_state), grad in zip(trajectory, policy_grad):
                policy.nn.head.weight.data[action, state] -= alpha * grad
                
        plot()

    def plot():
        with output:
            output.clear_output()
            value_function = linear_system_policy_evaluation(
                environment, policy, gamma)
            plot_value_and_policy(value_function.table.reshape(5, 5).detach().numpy(),
                                  policy.table.argmax(0).reshape(5, 5).detach().numpy(), 
                                  title_value="Value Evaluation",
                                  title_policy="Policy"
                                 )
            plt.show()

    button = ipywidgets.Button(description="Step 10 Episodes")
    button.on_click(lambda b: step(num_episodes=10))
    button2 = ipywidgets.Button(description="Step 100 Episodes")
    button2.on_click(lambda b: step(num_episodes=100))
    display(output, button, button2)
    plot()


interact(
    reinforce,
    gamma=ipywidgets.FloatSlider(
        value=0.9, min=0., max=0.99, step=1e-2, continuous_update=False),
    alpha=ipywidgets.FloatSlider(
        value=0.5, min=0., max=2.0, step=1e-2, continuous_update=False),
    baseline=False,
);

interactive(children=(FloatSlider(value=0.9, continuous_update=False, description='gamma', max=0.99, step=0.01…

## Demo Guide:

#### some explanation
This demo shows one of the basic reinforce policy gradient method REINFORCE. In this apporach we initialize the policy weight for the neural network which take the states as input and returns the probability of actions. So the initial policy is random. Secondly we use the policy to play N steps of the game — record action probabilities-from policy, reward-from environment, action — sampled by agent. Nextly we calculate the discounted reward for each step by backpropagation and calculate expected reward G. Finally we adjust weights of policy (back-propagate error in NN) to increase G.



- gamma: the discount factor of the environment.
- alpha: the learning rate in the Q learning
- baseline: use the baselines to further reduce the variance.

- step 10 Episode: Interacting the environment 1 time, generate one state, action, reward pair which has the length of horizon 20. 
- step 100 Episode: Interacting the environment 10 times, Note that the MDPs is learned after each iteration.

- Value Evaluation: the value evaluated by the policy learned by the REINFORCE.
- Policy: the optimal policy from the REINFORCE.


#### play around
- Run the REINFORCE with different alpha to see how fast it converges to the optimal policy or whether it stuck?
- See the difference between REINFORCE and REINFORCE baseline.


# REINFORCE w/ Function Approximation

In [8]:
def run(env_name, agent_name):
    if "CartPole" in env_name:
        max_steps = 200
    else:
        max_steps = 1000 
        
    environment = GymEnvironment(env_name)
    agent = getattr(
        importlib.import_module("rllib.agent"), 
        f"{agent_name}Agent"
    ).default(environment, num_iter=1, num_rollouts=1)
    
    try:
        train_agent(environment=environment, agent=agent, num_episodes=200, max_steps=max_steps, render=True, plot_flag=False)
    except KeyboardInterrupt:
        pass 
    environment.close()
    
    IPython.display.clear_output()
    
    plt.plot(agent.logger.get("train_return-0"), linewidth=16)
    plt.xlabel("Episode")
    plt.ylabel("Return");
    
interact(
    run,
    env_name = ["CartPole-v0", "Acrobot-v1", "MountainCar-v0"],
    agent_name = ["REINFORCE"]
);

interactive(children=(Dropdown(description='env_name', options=('CartPole-v0', 'Acrobot-v1', 'MountainCar-v0')…

## Demo Guide:

#### some explanation
This demo shows running the REINFORCE algorithm using neural network as approxiamtion on different enviornment.
- env_name: CartPole, Acrobot, MountainCar. Note that the Acrobot and MountainCar will take longer time for convergence.
- agent_name: REINFORCE.  

#### play around
- Play with the REINFORCE on different environment
- Compare the difference of the performance with Q-learning

# Actor Critic  

In [10]:
def actor_critic(gamma, alpha, eps):
    output = ipywidgets.Output()
    global policy, state, action 
    environment = EasyGridWorld()
    _, pi_star, _ = policy_iteration(environment, gamma, max_iter=10)
    pi_star = pi_star.table.argmax(0).reshape(5, 5).detach().numpy()
    torch.manual_seed(0)
    np.random.seed(0)
    
    policy = TabularPolicy(num_states=environment.num_states,
                           num_actions=environment.num_actions)
    critic = TabularQFunction(num_states=environment.num_states,
                              num_actions=environment.num_actions)
    nn.init.ones_(policy.nn.head.weight)
    
    state = environment.reset()

    if np.random.rand() < eps:
        action = np.random.choice(environment.num_actions)
    else:
        action = Categorical(
            logits=policy(torch.tensor(state).long())
        ).sample().item()
    def step(num_iter):
        global state, action
        for i in range(num_iter):
            q_val = critic(torch.tensor(state).long(), torch.tensor(action).long())

            next_state, reward, done, info = environment.step(action)
            
            if np.random.rand() < eps:
                next_action = np.random.choice(environment.num_actions)
            else:
                next_action = Categorical(
                    logits=policy(torch.tensor(next_state).long())
                ).sample().item()
            
            next_q = critic(torch.tensor(next_state).long(), torch.tensor(next_action).long())
            
            # update q function.
            reward = torch.tensor(reward).double()
            td = reward + gamma * next_q - q_val
            critic.set_value(state, action, q_val + alpha * td)
            
            # update policy, here directly log_p is parameterized with a tabular funciton. 
            grad = -q_val.detach()
            policy.nn.head.weight.data[action, state] -= alpha * grad[..., 0]
            state, action = next_state, next_action

        plot()
        
    def plot():
        with output:
            output.clear_output()
            def plot_(action):
                fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(20, 12))
                value_function = integrate_q(critic, policy)
                true_value_function = linear_system_policy_evaluation(environment, policy, gamma)

                vf = value_function.table.reshape(5, 5).detach().numpy()
                true_vf = true_value_function.table.reshape(5, 5).detach().numpy()
                pi = policy.table.argmax(0).reshape(5, 5).detach().numpy()

                if action == "best":
                    q_star = critic.table.max(0)[0].reshape(5, 5).detach().numpy()
                elif action == "down":
                    q_star = critic.table[0].reshape(5, 5).detach().numpy()
                elif action == "up":
                    q_star = critic.table[1].reshape(5, 5).detach().numpy()
                elif action == "right":
                    q_star = critic.table[2].reshape(5, 5).detach().numpy()
                elif action == "left":
                    q_star = critic.table[3].reshape(5, 5).detach().numpy()


                plot_value_function(vf, axes[0, 0], title="Estimated Value")
                plot_value_function(true_vf, axes[1, 0], title="True Value")

                plot_value_function(q_star, axes[0, 1], title="Estimated Critic")
                plot_value_function(q_star - vf, axes[1, 1], title="Estimated Advantage")

                plot_policy(pi, axes[0, 2], title="Current Policy")
                plot_policy(pi_star, axes[1, 2], title="Optimal Policy")

            interact(plot_,
                     action=ipywidgets.ToggleButtons(
                         options=["best", "down", "up", "left", "right"],
                         value='best',
                         description='Critic Action:',
                         style = {'description_width': 'initial'}
                     ))

    button = ipywidgets.Button(description="Step 100")
    button.on_click(lambda b: step(num_iter=100))
    button2 = ipywidgets.Button(description="Step 1000")
    button2.on_click(lambda b: step(num_iter=1000))
    display(output, button, button2)
    plot()


interact(
    actor_critic,
    gamma=ipywidgets.FloatSlider(
        value=0.9, min=0., max=0.99, step=1e-2, continuous_update=False),
    alpha=ipywidgets.FloatSlider(
        value=0.6, min=0., max=1.0, step=1e-2, continuous_update=False),
    eps=ipywidgets.FloatSlider(
        value=0.25, min=0., max=0.5, step=1e-2, continuous_update=False)
);

interactive(children=(FloatSlider(value=0.9, continuous_update=False, description='gamma', max=0.99, step=0.01…

## Demo Guide:

#### some explanation
This demo runs Actor Critic reinforcment learning approach:  In a simple term, Actor-Critic is a Temporal Difference(TD) version of Policy gradient. It has two networks: Actor and Critic. The actor decided which action should be taken and critic inform the actor how good was the action and how it should adjust. The learning of the actor is based on policy gradient approach. In comparison, critics evaluate the action produced by the actor by computing the value function.


- gamma: the discount factor of the environment.
- alpha: the learning rate in the Q learning
- eps: the parameter trading the exploration and exploitation. With epislon probability will pick random action and 1-epislon pick the best action.
- critic action:best, down, up, left, right: showthe estimate value of state-action pair.You have to choose the action first so given the action and states the critic will output the value function. 
- step 100: forward 100 steps and collect samples.
- step 1000: forward 1000 steps and collect samples.

- Estimated value: the estimated value function learned by interacting the world.
- Estimated Critic: Given the action the value function from the critics. The best means the argmax of critic actions.
- Current Policy: the optimal policy based on the learned by Actor-Critics
- True Value: The ground Truth of the value function calculated by linear system policy evalution given the current polict. Used as comparison of estimated value.
- Estimated Advantage: The advantage calculate the difference between value of state-action function and estimated value function.
- Optimal Policy: The ground Truth of the optimal policy. Used as comparison of current policy


#### play around
- Do 100 step of default setup and see how the estimated value and critic changes
- Change the critic action and see if it is reasonble for different action compared to the optimal policy action.
- Tune the parameter and run the demo to see the convergence of estimated value and true value.
- Check the change of estimated advantage, how does it change and when will it converge to 0?


# Actor Critic w/ Function Approximation

In [11]:
from rllib.util.losses import EntropyLoss
def run(env_name, agent_name, eta):
    if "CartPole" in env_name:
        max_steps = 200
    else:
        max_steps = 1000 
        
    environment = GymEnvironment(env_name, seed=0)
    agent = getattr(
        importlib.import_module("rllib.agent"), 
        f"{agent_name}Agent"
    ).default(environment, num_iter=1, num_rollouts=1)
    agent.algorithm.entropy_loss = EntropyLoss(
        eta=eta * 0.01, regularization=True
    )
    try:
        train_agent(environment=environment, agent=agent, num_episodes=200, max_steps=max_steps, render=True, plot_flag=False)
    except KeyboardInterrupt:
        pass 
    environment.close()
    
    IPython.display.clear_output()
    
    plt.plot(agent.logger.get("train_return-0"), linewidth=16)
    plt.xlabel("Episode")
    plt.ylabel("Return");
    
interact(
    run,
    env_name = ["CartPole-v0", "Acrobot-v1"],
    agent_name = ["GAAC", "A2C", "ActorCritic"],
    eta=ipywidgets.FloatSlider(value=0.2, min=0., max=1, step=0.1, continuous_update=False, description='entropy Reg:'),
);

interactive(children=(Dropdown(description='env_name', options=('CartPole-v0', 'Acrobot-v1'), value='CartPole-…

## Demo Guide:

#### some explanation
This demo runs Actor Critic reinforcment learning method with function approximation using neural network.
- env_name: CartPole, Acrobot, MountainCar. Note that the Acrobot and MountainCar will take longer time for convergence.
- agent_name: [generalized advantage actor critic(GAAC)](https://arxiv.org/abs/1506.02438), [Advantage-Actor Critic(A2C)](https://proceedings.mlr.press/v48/mniha16.html), ActorCritic. More details can be found within the link: 
- entropy Reg: regularization parameter eta in entropy loss.In regularization mode, it returns a loss given by:

$$ Loss = -\eta * entropy. $$
    
In trust-region mode (regularization=False), it returns a loss given by:
$$ Loss = \eta * (entropy - target\_entropy). $$

#### play around
Change the environment and agent name and see the difference.

# Constrained Policy Gradients

In [12]:
def run(env_name, agent_name):
    if "CartPole" in env_name:
        max_steps = 200
    else:
        max_steps = 1000 
        
    environment = GymEnvironment(env_name)
    agent = getattr(
        importlib.import_module("rllib.agent"), 
        f"{agent_name}Agent"
    ).default(environment)
    
    try:
        train_agent(environment=environment, agent=agent, num_episodes=200, max_steps=max_steps, render=True, plot_flag=False)
    except KeyboardInterrupt:
        pass 
    environment.close()
    
    IPython.display.clear_output()
    
    plt.plot(agent.logger.get("train_return-0"), linewidth=16)
    plt.xlabel("Episode")
    plt.ylabel("Return");
    
interact(
    run,
    env_name = ["CartPole-v0", "Acrobot-v1", "MountainCar-v0"],
    agent_name = ["TRPO", "PPO"]
);

interactive(children=(Dropdown(description='env_name', options=('CartPole-v0', 'Acrobot-v1', 'MountainCar-v0')…

## Demo Guide:

#### some explanation
This demo runs constrained policy gradients method. TPRO iteratively optimizes a asurrogates objective within trust region. PPO is an effective heuristic variant.
- env_name: CartPole, Acrobot, MountainCar. Note that the Acrobot and MountainCar will take longer time for convergence.
- agent_name: [Trust region policy optimization(TPRO)](https://proceedings.mlr.press/v37/schulman15.html), [Proximal policy optimization algorithms(PPO)](https://arxiv.org/abs/1707.06347). More details can be found within the link: 

#### play around
Change the environment and agent name and see the difference.


# Pathwise Policy Gradients

In [13]:
def run(env_name, agent_name):
    max_steps = 2500 
        
    environment = GymEnvironment(env_name)
    agent = getattr(
        importlib.import_module("rllib.agent"), 
        f"{agent_name}Agent"
    ).default(environment, exploration_episodes=10)
    agent.params["exploration_noise"] = ExponentialDecay(start=1.0, end=0.2, decay=1e5)
    agent.policy.dist_params.update(policy_noise = agent.params["exploration_noise"])
    try:
        train_agent(environment=environment, agent=agent, num_episodes=25, max_steps=max_steps, render=True, plot_flag=False)
    except KeyboardInterrupt:
        pass 
    environment.close()
    
    IPython.display.clear_output()
    
    plt.plot(agent.logger.get("train_return-0"), linewidth=16)
    plt.xlabel("Episode")
    plt.ylabel("Return");
    
interact(
    run,
    env_name = [ "MountainCarContinuous-v0", "Pendulum-v1"],
    agent_name = ["TD3", "SAC"]
);

interactive(children=(Dropdown(description='env_name', options=('MountainCarContinuous-v0', 'Pendulum-v1'), va…

## Demo Guide:

#### some explanation
This demo runs off policy polict gradient algorithms. TD3 is extension of DDPG to avoid maximization bias. SAC method is variant of TD3 for entropy regualized MDPs. More details could be checked in the following link.


- env_name: MountainCarContinuous-v0, Pendulum-v0, HalfCheetah-v2.
- agent_name: [Trust region policy optimization(TPRO)](https://proceedings.mlr.press/v37/schulman15.html), [Proximal policy optimization algorithms(PPO)](https://arxiv.org/abs/1707.06347). More details can be found within the link: 

#### play around
Change the environment and agent name and see the difference.
